# S3W11D3: 从“脚本”到“系统” (Engineering Refactor)

## 🎯 今日目标 (Today's Goals)

1. **配置中心化**: 创建 `src/config.py`，告别硬编码 (Magic Numbers)。
2. **核心封装**: 创建 `src/rag/engine.py`，实现 `RAGPipeline` 类，统一管理“入库”和“检索”流程。
3. **算法攻坚**: 掌握 BST 的增删操作，特别是 **LC 450 (删除节点)** 的三种情况处理。

## 1. 工程重构：配置中心化 (`src/config.py`)

**第一性原理**: **Single Source of Truth (单一真理来源)**。
不要在代码里写 `chunk_size=500` 或者 `top_k=3`。如果明天你想改成 5，难道要搜遍全项目去改吗？

请创建/更新 `src/config.py`:

```python
# src/config.py
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

class Config:
    # --- 基础路径 ---
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    DATA_DIR = os.path.join(PROJECT_ROOT, "data")
    CHROMA_DB_DIR = os.path.join(DATA_DIR, "chroma_db_data")
    
    # --- LLM 配置 ---
    LLM_API_KEY = os.getenv("DEEPSEEK_API_KEY") or os.getenv("SILICONFLOW_API_KEY")
    LLM_BASE_URL = os.getenv("DEEPSEEK_BASE_URL") or "https://api.siliconflow.cn/v1"
    LLM_MODEL_NAME = "deepseek-chat"
    
    # --- RAG 参数 ---
    EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    CHUNK_SIZE = 500       # 每个切片的大小
    CHUNK_OVERLAP = 50     # 切片重叠部分
    RETRIEVAL_TOP_K = 3    # 每次检索几条
    
    # --- Prompt 模板 ---
    RAG_SYSTEM_PROMPT = """
    你是一个基于知识库的智能助手。请严格根据以下【参考文档】回答用户问题。
    如果参考文档中没有答案，请直接说"不知道"，不要编造。
    
    【参考文档】:
    {context}
    """

# 实例化，方便其他模块直接 import config
config = Config()

```



## 2. 工程重构：RAG 核心引擎 (`src/rag/engine.py`)

这是我们 RAG 系统的“主板”。它负责把 Embedding、VectorDB 和 LLM 连接起来。

请创建/更新 `src/rag/engine.py`:

```python
# src/rag/engine.py
import os
from typing import List
# 引入配置
from src.config import config
# 引入之前的组件 (假设你之前已经写好了这些，如果没有，需要把 Week 9/10 的代码搬运过来)
# 这里为了演示完整性，我使用了伪代码导入，你需要根据你的实际文件路径调整
from src.rag.vector_db import VectorDBHandler  # 假设你封装了 Chroma 操作
from src.llm.client import LLMClient           # 假设你封装了 OpenAI Client
from src.rag.embedding import EmbeddingModel   # 假设你封装了 SentenceTransformer

class RAGPipeline:
    def __init__(self):
        """初始化 RAG 流水线的所有组件"""
        print("⚙️ 初始化 RAG Pipeline...")
        
        # 1. 加载 Embedding 模型
        self.embedding_model = EmbeddingModel(model_name=config.EMBEDDING_MODEL)
        
        # 2. 连接向量数据库
        self.vector_db = VectorDBHandler(
            persist_directory=config.CHROMA_DB_DIR,
            embedding_fn=self.embedding_model
        )
        
        # 3. 初始化 LLM 客户端
        self.llm = LLMClient(
            api_key=config.LLM_API_KEY,
            base_url=config.LLM_BASE_URL,
            model=config.LLM_MODEL_NAME
        )
    
    def ingest_documents(self, file_path: str):
        """
        [ETL] 数据入库流程: 读取 -> 切分 -> 向量化 -> 存储
        """
        print(f"📥 正在处理文件: {file_path}")
        # 1. 读取 & 切分 (这里可以使用 LangChain 的 Splitter，也可以手写)
        # chunks = load_and_split(file_path, config.CHUNK_SIZE, config.CHUNK_OVERLAP)
        
        # 暂用伪代码代替
        chunks = ["这是测试文档段落1...", "这是测试文档段落2..."] 
        
        # 2. 存入向量库
        self.vector_db.add_texts(chunks)
        print(f"✅ 入库完成，共 {len(chunks)} 个片段。")

    def query(self, user_query: str) -> str:
        """
        [Inference] 检索 + 生成流程
        """
        print(f"🔍 用户提问: {user_query}")
        
        # 1. 检索 (Retrieve)
        # vector_db 内部会自动把 query 变成向量
        relevant_docs = self.vector_db.search(
            query=user_query, 
            top_k=config.RETRIEVAL_TOP_K
        )
        
        # 2. 构建上下文 (Context Construction)
        context_str = "\n\n".join(relevant_docs)
        
        # 3. 生成 Prompt (Augment)
        system_prompt = config.RAG_SYSTEM_PROMPT.format(context=context_str)
        
        # 4. 调用 LLM (Generate)
        answer = self.llm.chat_completion(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )
        
        return answer

# 单例模式：创建一个全局可用的 pipeline 实例
# 这样下次 import rag_engine 时不需要重新加载模型
rag_engine = RAGPipeline()

```

**✅ 动作**:

1. 确保你的 `src/rag/vector_db.py` 等文件存在且可用。
2. 如果有缺失的模块，今天的任务就是**把 Notebook 里的代码搬运进去**。
3. 执行 `git add .` 和 `git commit -m "refactor: implement RAGPipeline and centralized config"`。

## 3. 算法攻坚：BST 的增删 (The Surgical Operations)

如果说昨天的“验证 BST”是**只读**操作，今天的“增删”就是**写**操作。

### LC 701. 二叉搜索树中的插入操作 (Insert)

* **难度**: Medium (其实是 Easy)
* **思路**:
* 比根小？往左走。左边没人？你就在这安家。
* 比根大？往右走。右边没人？你就在这安家。


* **代码** (直接写在 LeetCode 上):
```python
def insertIntoBST(self, root: Optional[TreeNode], val: int) -> Optional[TreeNode]:
    if not root:
        return TreeNode(val)
    if val < root.val:
        root.left = self.insertIntoBST(root.left, val)
    else:
        root.right = self.insertIntoBST(root.right, val)
    return root

```

### 🔥 LC 450. 删除二叉搜索树中的节点 (Delete) - **今日重难点**

删除节点比插入难得多，因为删掉一个节点后，可能会留下一个“坑”，我们需要找人来填这个坑，还要保持 BST 的性质（左小右大）。

**三种情况 (Case Analysis)**:

1. **叶子节点**: 直接删，没后顾之忧。
2. **只有一个孩子**: 父爱如山，让孩子直接顶替自己的位置。
3. **有两个孩子 (Trouble!)**:
* 比如你要删掉根节点 `5`，它左边有 `3`，右边有 `7`。谁来当新的王？
* **继承法则**: 只有两个人有资格继承皇位：
* **左派领袖 (Predecessor)**: 左子树里**最大**的那个。
* **右派领袖 (Successor)**: 右子树里**最小**的那个。


* **通用做法**: 找到 **右子树里最小的节点 (Successor)**，把它的值复制给当前节点，然后把那个 Successor 删掉。



#### 代码实现 (标准模板)

```python
class Solution:
    def deleteNode(self, root: Optional[TreeNode], key: int) -> Optional[TreeNode]:
        if not root:
            return None
        
        # 1. 先找节点
        if key < root.val:
            root.left = self.deleteNode(root.left, key)
        elif key > root.val:
            root.right = self.deleteNode(root.right, key)
        else:
            # 2. 找到了，开始处理删除逻辑
            
            # Case 1 & 2: 只有一个孩子或没有孩子
            if not root.left:
                return root.right
            if not root.right:
                return root.left
            
            # Case 3: 两个孩子都在
            # 策略: 找右子树中最小的节点 (Successor) 来顶替
            successor = self.get_min(root.right)
            
            # 这里的操作很灵性：不是移动节点对象，而是直接“夺舍”（替换值）
            root.val = successor.val
            
            # 既然值已经拿上来了，那个 successor 也没用了，去右子树里把它删掉
            root.right = self.deleteNode(root.right, successor.val)
            
        return root

    def get_min(self, node):
        # 一直往左走，最左边的就是最小的
        while node.left:
            node = node.left
        return node

```



## 4. 今日任务总结

1. [ ] **Engineering**: 成功创建 `src/config.py` 和 `src/rag/engine.py`。
* *自测*: 能够在一个新的 Python 文件里 `from src.rag.engine import rag_engine` 并不报错。


2. [ ] **Algorithm**: 深刻理解并 AC LC 450 (删除节点)。**一定要自己画图理解“继任者”逻辑**。
3. [ ] **Git**: 提交一个干净的 Refactor Commit。

💡 **导师提示**:
今天的重构可能比较枯燥，因为都是在搬运代码。但请相信我，**如果不做这一步，你的项目在面试官眼里就是一个“学生作业”；做了这一步，它就是一个“工程项目”。**

加油！如果配置路径遇到问题（比如 `ModuleNotFoundError`），随时把报错发给我。